In [ ]:
from flask import Flask, jsonify
import json
import psycopg2

app = Flask(__name__)

# PostgreSQL database connection settings
DB_HOST = 'your_host'
DB_PORT = 'your_port'
DB_NAME = 'your_database_name'
DB_USER = 'your_username'
DB_PASSWORD = 'your_password'

# Load the JSON data from the file
def load_data():
    with open('housing_data.json') as file:
        data = json.load(file)
    return data

# Create the houses table in the database
def create_table():
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USER, password=DB_PASSWORD)
    cursor = conn.cursor()

    # Create the table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS houses (
            id SERIAL PRIMARY KEY,
            location VARCHAR(255),
            sale_price NUMERIC
        );
    ''')
    conn.commit()
    conn.close()

# Insert the JSON data into the database
def insert_data(data):
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USER, password=DB_PASSWORD)
    cursor = conn.cursor()

    # Insert each house into the table
    for house in data:
        location = house['location']
        sale_price = house['sale_price']
        cursor.execute('INSERT INTO houses (location, sale_price) VALUES (%s, %s)', (location, sale_price))
    
    conn.commit()
    conn.close()

# Calculate the average sale price of all houses
def calculate_average_price_overall():
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USER, password=DB_PASSWORD)
    cursor = conn.cursor()

    # Execute the SQL query to calculate the average sale price
    cursor.execute('SELECT AVG(sale_price) FROM houses')
    average_price = cursor.fetchone()[0]

    conn.close()
    return average_price

# Calculate the average sale price of houses per location
def calculate_average_price_per_location():
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USER, password=DB_PASSWORD)
    cursor = conn.cursor()

    # Execute the SQL query to calculate the average sale price per location
    cursor.execute('SELECT location, AVG(sale_price) FROM houses GROUP BY location')
    result = cursor.fetchall()
    average_prices = {location: price for location, price in result}

    conn.close()
    return average_prices

# Get the maximum sale price
def get_max_sale_price():
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USER, password=DB_PASSWORD)
    cursor = conn.cursor()

    # Execute the SQL query to get the maximum sale price
    cursor.execute('SELECT MAX(sale_price) FROM houses')
    max_price = cursor.fetchone()[0]

    conn.close()
    return max_price

# Get the minimum sale price
def get_min_sale_price():
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USER, password=DB_PASSWORD)
    cursor = conn.cursor()

    # Execute the SQL query to get the minimum sale price
    cursor.execute('SELECT MIN(sale_price) FROM houses')
    min_price = cursor.fetchone()[0]

    conn.close()
    return min_price

# Define API endpoints

@app.route('/api/average_price/overall')
def average_price_overall():
    average_price = calculate_average_price_overall()
    return jsonify({'average_price': average_price})

@app.route('/api/average_price/location')
def average_price_per_location():
    average_prices = calculate_average_price_per_location()
    return jsonify({'average_prices': average_prices})

@app.route('/api/max_price')
def max_price():
    max_price = get_max_sale_price()
    return jsonify({'max_price': max_price})

@app.route('/api/min_price')
def min_price():
    min_price = get_min_sale_price()
    return jsonify({'min_price': min_price})

if __name__ == '__main__':
    # Load the JSON data
    data = load_data()

    # Create the houses table in the database
    create_table()

    # Insert the JSON data into the database
    insert_data(data)

    # Run the Flask application
    app.run()
